In [2]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
import jReversion as jR
from LDOI import BooleanDOI_processing as BDOIp
from LDOI import qm
import pandas as pd
import numpy as np
import networkx as nx
import concurrent.futures
import par_helper as ph
import random
import robustness as rb
import pickle

In [2]:
networkModel =[ 'bortezomib',
                # 'igvh',
                'apoptosis',
                # 'aurora',
                #'bt474_long',
                'bt474_short',
                # 'cd4t',
                'colitis',
                'death',
                # 'egfr',
                # 'erbb',
                # 'fa_brca',
                # 'fa_check',
                #'hcc1954_long',
                'hcc1954_short',
                'hgf',
                'mammalian',
                # 'mammalian_2006',
                'mapk',
                'oxidative',
                # 'pro_inflammatory',
                'fibroblasts',
                #'skbr3_long',
                'skbr3_short',
                'tlgl_2008',
                'tlgl_2011',
                # 'tlgl_2011_reduced',
                # 'prostate',
                'migration']

In [3]:
def r3_test(dir_path, model_name, net_idx, prefix='n', suffix='n', sampling_num=10):
    with open(dir_path + model_name + '_' + str(net_idx) + '.pck', 'rb') as f:  # notice the r instead of w
        g = pickle.load(f)
    
    read_nodes = list(g.nodes)
    mapping = {}  # nodename to number index
    inverse_mapping = {}  # number index to nodename
    read_nodes_dict = {}
    inverse_read_nodes_dict = {}
    for i, node in enumerate(read_nodes):
        index = prefix + str(i) + suffix
        mapping[str(node)] = index
        inverse_mapping[index] = str(node)
        mapping['~' + str(node)] = '~' + index
        inverse_mapping['~' + index] = '~' + str(node)
        read_nodes_dict[i] = str(node)
        inverse_read_nodes_dict[str(node)] = i
    
    input_nodes = [node for node in g.nodes if g.in_degree(node) == 0]
    output_nodes = [node for node in g.nodes if g.out_degree(node) == 0]
    
    num_inputs = len(input_nodes)
    num_input_conditions = 2 ** num_inputs
    input_conditions = np.ndarray((num_inputs, 2), dtype=object)
    for idx, input_node in enumerate(input_nodes):
        input_conditions[idx, 0] = '~' + str(input_node)
        input_conditions[idx, 1] = str(input_node)
    
    
    output_nodes_ex = []
    output_nodes_ex.extend([read_nodes_dict[idx] for idx in output_nodes])
    output_nodes_ex.extend(['~'+read_nodes_dict[idx] for idx in output_nodes])
    input_nodes_ex = input_conditions.reshape(num_inputs * 2, ).tolist()
    
    try:
        GExpanded = nx.read_gml('/data2/jijoo/theor/exnet/' + model_name + '_' + str(net_idx) + '_ex.gml')
    except:
        #     GExpanded = ph.par_get_expanded_network(g, prefix=prefix, suffix=suffix, worker=num_worker)
        GExpanded = BDOIp.Get_expanded_network(g, prefix=prefix, suffix=suffix)
        nx.write_gml(GExpanded, '/data2/jijoo/theor/exnet/' + model_name + '_' + str(net_idx) + '_ex.gml')
        
    TempGIOW = jR.get_input_output_relation(GExpanded, mapping, inverse_mapping, input_conditions, output_nodes_ex,
                                            constant_nodes=[])
    LDOIs = TempGIOW['LDOIs']
    GeneLDOIs = TempGIOW['gene_LDOIs']
    Conflicts = TempGIOW['conflicts']
    GeneConflicts = TempGIOW['gene_conflicts']
    IORelation = TempGIOW['io_relation']
    GRemained = TempGIOW['G_remained']

    R0 = jR.identifying_r0_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
    # DM = jR.identifying_disconnecting_mutations(GExpanded, InputNodes, OutputNodes, Mapping, InverseMapping)
    R1 = jR.identifying_r1_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
#     IIDC = jR.identifying_input_independent_canalizing_mutations(GExpanded, OutputNodes, Mapping, InverseMapping)
#     UN = jR.identifying_input_unreachable_nodes(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions)
    RN = jR.identifying_rn_mutations(GExpanded, mapping, inverse_mapping, input_nodes_ex, output_nodes_ex, input_conditions,
                                     IORelation,
                                     R0['ineffective_mutations'], R1['r1_mutations'])
    
    NodeList = set(read_nodes_dict.values())
    NodeList.difference_update(input_nodes)
    NodeList.difference_update(output_nodes)
    C0, C1, C2, C3 = [], [], [], []
    for NODE in NodeList:
        negNODE = '~' + NODE
        if R0['ineffective'][NODE] and R0['ineffective'][negNODE]:
#             nodeClass = 'C0'
            C0.append(NODE)
        elif NODE in RN['rn_mutations']:
            if RN['rn_mutations'][NODE] == 'R1':
#                 nodeClass = 'C1'
                C1.append(NODE)
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
            else:
#                 nodeClass = 'C2'
                C2.append(NODE)
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
        elif negNODE in RN['rn_mutations']:
            if RN['rn_mutations'][negNODE] == 'R1':
#                 nodeClass = 'C1'
                C1.append(NODE)
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
        else:
#             nodeClass = 'C3'
            C3.append(NODE)
#             canalizing = IIDC['iid_canalizing'][NODE]
#             unreachable = UN['input_unreachable'][NODE]
    

    robustness_pa = rb.robustness_primary_attractor(g_read=g, state_num=2**sampling_num)
    robustness_ip = rb.robustness_initial_perturbation(g_read=g, state_num=2**sampling_num)
    
    robustness = []
    for key, val in robustness_pa.items():
        robustness.append(val['oe'])
        robustness.append(val['ko'])
#     print(np.mean(robustness))
    
    
    # deterministic_IO
    empty_io = [x for x in IORelation[0,:] if len(x) == 0]
    len_io = [len(x) for x in IORelation[0,:]]
    empty_io_ratio = len(empty_io) / num_input_conditions
    deterministic_io_ratio = sum(len_io) / (num_input_conditions * len(output_nodes))
    
    
    LDOI_union = set()
    for LDOI in LDOIs[0,:]:
        LDOI_union = LDOI_union.union(LDOI)
    LDOI_union = LDOI_union.union([mapping[x] for x in input_conditions.reshape((2*num_inputs,))])
    GSub = nx.subgraph(GExpanded, LDOI_union)
    io_count = 0
    io_pathway_count = 0
    for i in range(num_input_conditions):
        # G_copied = G_expanded.copy()
        BinaryBit = np.binary_repr(i, width=num_inputs)
        InputCondition = [input_conditions[x, int(BinaryBit[x])] for x in range(num_inputs)]
        # source_nodes = input_condition + constant_nodes
        Sources = set([x for x in InputCondition])
        # for node in source:
        #     G_copied.remove_edges_from(list(G_copied.in_edges(node)))
        #     G_copied.remove_edges_from(list(G_copied.in_edges(BDOId.Negation_in_expanded(node))))

        Targets = set([x for x in IORelation[0, i]])
        if len(Targets) > 0:
            for s in Sources:
                for t in Targets:
                    if nx.has_path(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')]):
                        io_count += 1
                        len_shortest_path = nx.shortest_path_length(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')])
                        for p in nx.all_simple_paths(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')], cutoff=2*len_shortest_path):
                            io_pathway_count += 1

    if io_count == 0:
        redundancy = -1
    else:
        redundancy = io_pathway_count/float(io_count)
    
    return {'size_of_network': len(g), 'num_link': len(g.edges), 'num_input': len(input_nodes), 'num_output': len(output_nodes),
            'C0': len(C0), 'C1': len(C1), 'C2': len(C2), 'C3': len(C3), 
            'robustness_pa': np.mean(robustness), 'robustness_ip': robustness_ip,
            'empty_io_ratio': empty_io_ratio, 'deterministic_io_ratio':deterministic_io_ratio,
            'redundancy': redundancy}

In [6]:
result_pd = pd.DataFrame(columns=['model', 'network_idx', 'size_of_network', 'num_link', 'C0', 
                              'C1', 'C2', 'C3', 'robustness_pa', 'robustness_ip', 'empty_io_ratio', 'deterministic_io_ratio', 
                              'redundancy'])

In [1]:
networkModel =[ #'bortezomib',
                # 'igvh',
#                 'apoptosis',
                # 'aurora',
                #'bt474_long',
#                 'bt474_short',
                # 'cd4t',
#                 'colitis',
#                 'death',
                # 'egfr',
                # 'erbb',
                # 'fa_brca',
                # 'fa_check',
                #'hcc1954_long',
#                 'hcc1954_short',
#                 'hgf',
#                 'mammalian',
                # 'mammalian_2006',
#                 'mapk',
#                 'oxidative',
                # 'pro_inflammatory',
                'fibroblasts',
                #'skbr3_long',
                'skbr3_short',
                'tlgl_2008',
                'tlgl_2011',
                # 'tlgl_2011_reduced',
                # 'prostate',
                'migration']

In [ ]:
for Model in networkModel:
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(r3_test, dir_path='../networks/configuration_models/', model_name=Model, net_idx=network_idx, prefix='n', suffix='n', sampling_num=10): network_idx for network_idx in range(100)}

    for future in concurrent.futures.as_completed(futures):
        network_idx = futures[future]
        temp = future.result()
        temp['network_idx'] = network_idx
        temp['model'] = Model
        result_pd = result_pd.append(temp, ignore_index=True)
    
    print(Model + ': DONE')
# configuration_models

In [7]:
result_pd

model network_idx size_of_network num_link  C0  C1 C2 C3  \
0    bortezomib          61              67      126  56   8  0  3   
1    bortezomib          27              67      128  65   0  0  2   
2    bortezomib          60              67      128  55   9  0  3   
3    bortezomib          26              67      127  50  14  1  2   
4    bortezomib          25              67      128  53  10  0  4   
..          ...         ...             ...      ...  ..  .. .. ..   
995   oxidative          81              19       30  19   0  0  0   
996   oxidative          11              19       30  17   2  0  0   
997   oxidative          80              19       30  19   0  0  0   
998   oxidative           0              19       30  17   2  0  0   
999   oxidative          48              19       30  19   0  0  0   

     robustness_pa  robustness_ip  empty_io_ratio  deterministic_io_ratio  \
0         0.656716       0.892141             1.0                0.000000   
1         0.731343       0.925373             1.0                0.000000   
2         0.636816       0.876720             0.5                0.166667   
3         0.738806       0.925373             1.0                0.000000   
4         0.675373       0.887608             1.0                0.000000   
..             ...            ...             ...                     ...   
995       0.894737       0.894737             1.0                0.000000   
996       0.736842       0.947368             1.0                0.000000   
997       0.671053       0.837068             1.0                0.000000   
998       0.723684       0.618575             1.0                0.000000   
999       0.763158       0.947368             1.0                0.000000   

    redundancy  num_input  num_output  
0           -1        5.0         3.0  
1           -1        5.0         3.0  
2         32.0        5.0         3.0  
3           -1        5.0         3.0  
4           -1        5.0         3.0  
..         ...        ...         ...  
995         -1        1.0         1.0  
996         -1        1.0         1.0  
997         -1        1.0         1.0  
998         -1        1.0         1.0  
999         -1        1.0         1.0  

[1000 rows x 15 columns]

In [8]:
result_pd.to_csv('../data/cc_config_211126.csv')